In [1]:
import PollingSystem as ps
import StaticRP as srp
import MarkovianRP as mrp
import dynamic_rp as drp

import numpy as np

In [2]:
#set up a simple, symmetric polling system
sys_traffic = 0.1
beta = 1
lambdas = np.array([1/3, 1/3, 1/3])*sys_traffic/beta
sym_sys = ps.PollingSystem(lambdas, beta)
S = np.array([[0, 3, 3],
              [3, 0, 3],
              [3, 3, 0]])

In [3]:
pi, w_rnd = sym_sys.calc_optimal_rp(S, rp_type = 0)
print(pi, w_rnd)
static_rand = mrp.RandomRP(pi)
P, w_mrkv = sym_sys.calc_optimal_rp(S, rp_type = 1)
print(P, w_mrkv)
static_markov = mrp.MarkovianRP(P)

[0.33333333 0.33333333 0.33333333] 6.000000000000001


/home/win/.local/lib/python3.8/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


[[0.00000000e+00 5.96448012e-04 9.99403552e-01]
 [9.99357140e-01 0.00000000e+00 6.42860424e-04]
 [8.67015555e-04 9.99132984e-01 0.00000000e+00]] 4.891229361880553


In [4]:
table_policy = srp.MinSSRPFromPis(pi, S)
cyclic_policy = srp.StaticRP(np.array([0,1,2]))

In [5]:
simple_rule = drp.DPRP(lambdas, beta, S, drp.shortest_lengths)
simple_rule_time_diffs = drp.DPRP(lambdas, beta, S, drp.shortest_lengths, fully_observed = False)

In [6]:
#now simulate and compare
t = 2*60*60
xt1, wt1, _, _, _, _, _  = sym_sys.simulate(static_rand, S, t)
print(wt1[-1][1])
xt2, wt2, _, _, _, _, _  = sym_sys.simulate(static_markov, S, t)
print(wt2[-1][1])
xt3, wt3, _, _, _, _, _  = sym_sys.simulate(table_policy, S, t)
print(wt3[-1][1])
xt4, wt4, _, _, _, _, _  = sym_sys.simulate(cyclic_policy, S, t)
print(wt4[-1][1])
xt5, wt5, _, _, _, _, _  = sym_sys.simulate(simple_rule, S, t)
print(wt5[-1][1])
xt6, wt6, _, _, _, _, _  = sym_sys.simulate(simple_rule_time_diffs, S, t, fully_observed = False)
print(wt6[-1][1])

6.130351785655419
5.021332150830562
4.911448810566129
4.894449480339737
3.938393616062812
4.964796373717104


In [12]:
#now let's look at a nice little MCTS

K = 7200
T = K*beta#(Two hours, since beta = 1)
print("Total time: %.2f minutes"%(T/60))


class PSState:
    def __init__(self, q, q_lengths, k):
        self.q = q
        self.q_lengths = q_lengths
        self.k = k
        
    def __eq__(self, other):
        #todo some sanity cheques
        return self.q == other.q and all(self.q_lengths == other.q_lengths) and self.k = other.k
    
class MCTSNode:
    
    def __init__(self, state):
        self.state = state
        #at decision points, queue q will be empty, don't allow it as a choice
        self.actions = [i for i in range(len(state.q_lengths)) if i != state.q]
        self.a_vists = {}
        self.visits = 0
        #map from actions to value
        self.est_future_vals = {}#plug in expected value of integral if possible
        self.stage_vals = {}#plug in expected value of integral if possible
        #map from child actions to lists of possible children (there will be multiple per action)
        self.children = {}
        
    def getUCB1Action(self):
        #todo - handle unvisited if numpy can't
        #TODO - double check that just making this a minus is ok when we're minimizing
        ucb_vals = [self.est_values[a] - np.sqrt(2*np.log(self.n_vists)/self.a_vists[a]) for a in self.actions]
        argmin = np.argmin(ucb_vals)
        return self.actions[argmin]
    
    def addFutureReward(a, r):
        n = self.a_visits[a]
        current_est = self.est_future_vals[a]
        self.est_future_vals[a] = (n*current_est + r)/(n+1)
        self.a_visits[a] += 1
        
class MCTSTree:
    
    def __init__(self, root, ps, S, K):
        self.root = root
        self.ps
        self.S
        #eventually, should probably use a better data structure
        #TODO intialize states to empty maps
        self.nodes = {}
        self.nodes[root.state.q] = [root]
        self.K = k
        
    def nextNode(self):
        node = self.root
        ex_node = None
        while not found_unexpanded_node:
            next_action = node.getUCB1Action()
            path.append( (node, next_action) )
            state  = node.state
            q_next, new_q_lengths, time = self.ps.stageTransitionRealization(state.q, state.q_lengths, next_action, self.S)
            #check if this state is in out set of states
            new_state = PSState(q_next, new_q_lengths, state.k + time//ps.beta)
            #first check if it's in the nodes set of children
            child_states = [mctsnode.state for mctsnode in node.children[next_action]]
            if new_state in child_states:
                nidx = child_states.index(new_state)
                node = node.children[next_action][nidx]
            #check if it's somewhere else to for more efficient use of simulations
            elif new_state in [mctsn.state for mctsn in self.nodes[q_next]]:
                
                nidx = [mctsn.state for mctsn in self.nodes[q_next]].index(new_state)
                nxt_node = self.states[q_next][nidx]
                node.children[next_action].append(nxt_node)
                node = nxt_node
            #ok we haven't seen this before
            else:
                found_unexpanded_node = True
                nxt_node = MCTSNode(new_state)
                node.children[next_action].append(nxt_node)
                self.nodes[q_next].append(nxt_node)
                node = nxt_node
        return path, ex_node
            
    def expand(self, ex_node, dflt_policy):
        state = ex_node.state
        #TODO, handle non-zero initial queue lengths
        #handle fact that first stage of the simulation
        #, its epxectation can be computed explicitly, so we leave it out
        # so we actually want to select a random state we arrive at after switching
        #and start simulations from there
        q_next = dflt_policy.next(state.q, state.q_lengths)
        q_start, lens_start, time = self.ps.stageTransitionRealization(state.q, state.q_lengths, q_next, self.S)
        xt, _, _, _, _, _, _ = self.ps.simulate(dflt_policy, self.S,
                                beta*(self.K) - time, q = q_start,
                                q_lengths = lens_start )
        int_L = MCTSTree.integrate_q_lengths(xt)
        return int_L, q_next
        
    def backProp(self, val, path):
        future_reward = val
        for (node, action ) in path[::-1]:
            node.addFutureReward(action, future_reward)
            future_reward += node.stage_vals[action]
    
    def integrate_q_lengths(xt):
        int_L = 0
        xt = np.array(xt)
        times = xt[0]
        total_length = np.sum(xt[2:], axis = 1)
        #todo - double check this matches how xt is created
        return total_length[:-1] @ (times[1:] - times[:-1])
        
x0 = PSState(0, np.zeros(3), 0)
root = MCTSNode(x0)
tree = MCTSTree(root)

while not converged:
    #find next node to expand
    path, ex_node = tree.nextNode()
    #rollout
    val, action = tree.expand(ex_node, dflt_policy)
    path.append((ex_node, action))
    tree.backProp(val, path)

Total time: 120.00 minutes


1